In [1]:
from pymongo import MongoClient
import pandas as pd
from pandas.io.json import json_normalize

pd.set_option('max_columns', 50)

In [6]:
# conexión con  mongo
client = MongoClient("mongodb://localhost:27017/")

# conexión con la base de datos "db_companies"
db = client.db_companies

#coleción companies
collection_companies = db.companies

# query pidiendo TODO
query = collection_companies.find()

In [7]:
# convertimos a pandas.DataFrame
df = pd.DataFrame(query)

# vemos el resultado
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,deadpooled_day,deadpooled_month,deadpooled_url,deadpooled_year,description,email_address,external_links,founded_day,founded_month,founded_year,funding_rounds,homepage_url,image,investments,ipo,milestones,name,number_of_employees,offices,overview,partners,permalink,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297d8b,None,[],Zoho ManageEngine,,,enterprise,[],2007-05-25 19:24:22,http://www.crunchbase.com/company/adventnet,NaN,NaN,NaN,2.0,Server Management Software,pr@adventnet.com,[],NaN,NaN,1996.0,[],http://adventnet.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,[],AdventNet,600.0,"[{'description': 'Headquarters', 'address1': '...","<p>AdventNet is now <a href=""/company/zoho-man...",[],abc3,925-924-9500,[],"[{'title': 'DHFH', 'is_past': True, 'provider'...","[{'is_past': True, 'title': 'CEO and Co-Founde...","[{'available_sizes': [[[150, 94], 'assets/imag...",,$0,manageengine,Wed Oct 31 18:26:09 UTC 2012,[]
1,52cdef7c4bab8bd675297d8a,"{'price_amount': 30000000, 'price_currency_cod...",[],,http://digitalquarters.net/feed/,http://digitalquarters.net/,web,"[{'competitor': {'name': 'Wikia', 'permalink':...",2007-05-25 06:51:27,http://www.crunchbase.com/company/wetpaint,NaN,NaN,NaN,1.0,Technology Platform Company,info@wetpaint.com,[{'external_url': 'http://www.geekwire.com/201...,17.0,10.0,2005.0,"[{'id': 888, 'round_code': 'a', 'source_url': ...",http://wetpaint-inc.com,"{'available_sizes': [[[150, 75], 'assets/image...",[],NaN,"[{'id': 5869, 'description': 'Wetpaint named i...",Wetpaint,47.0,"[{'description': '', 'address1': '710 - 2nd Av...",<p>Wetpaint is a technology platform company t...,[],abc2,206.859.6300,"[{'name': 'Wikison Wetpaint', 'permalink': 'we...",[],"[{'is_past': False, 'title': 'Co-Founder and V...","[{'available_sizes': [[[150, 86], 'assets/imag...","wiki, seattle, elowitz, media-industry, media-...",$39.8M,BachelrWetpaint,Sun Dec 08 07:15:44 UTC 2013,[]
2,52cdef7c4bab8bd675297d8c,None,[],,http://blogs.zoho.com/feed,http://blogs.zoho.com/,software,"[{'competitor': {'name': 'Empressr', 'permalin...",Fri May 25 19:30:28 UTC 2007,http://www.crunchbase.com/company/zoho,NaN,NaN,NaN,3.0,Online Business Apps Suite,info@zohocorp.com,[{'external_url': 'http://www.online-tech-tips...,15.0,9.0,2005.0,[],http://zoho.com,"{'available_sizes': [[[150, 55], 'assets/image...",[],NaN,"[{'id': 388, 'description': 'Zoho Reaches 2 Mi...",Zoho,1600.0,"[{'description': 'Headquarters', 'address1': '...","<p>Zoho offers a suite of Business, Collaborat...",[],abc4,1-888-204-3539,"[{'name': 'Zoho Office Suite', 'permalink': 'z...",[],"[{'is_past': False, 'title': 'CEO and Founder'...",[],"zoho, officesuite, spreadsheet, writer, projec...",$0,zoho,Wed Oct 30 00:07:05 UTC 2013,"[{'embed_code': '<object width=""430"" height=""2..."
3,52cdef7c4bab8bd675297d8f,None,[],None,http://feeds.feedburner.com/omnidrive,http://www.omnidrive.com/blog,network_hosting,"[{'competitor': {'name': 'Dropbox', 'permalink...",Sun May 27 03:25:32 UTC 2007,http://www.crunchbase.com/company/omnidrive,15.0,9.0,,2008.0,None,info@omnidrive.com,[],1.0,11.0,2005.0,"[{'id': 225, 'round_code': 'angel', 'source_ur...",http://www.omnidrive.com,"{'available_sizes': [[[150, 85], 'assets/image...",[],None,[],Omnidrive,NaN,"[{'description': '', 'address1': 'Suite 200', ...","<p>Currently in public beta, Omnidrive makes i...",[],omnidrive,660-675-5052,"[{'name': 'Omnidrive', 'permalink': 'omnidrive'}]",[],"[{'is_past': True, 'title': 'Co-founder', 'per...",[],"storage, sharing, edit, online",$800k,Nomadesk,Tue Jul 02 22:48:04 UTC 2013,[]
4,52cdef7c4bab8bd675297d8d,"{'price_amount': 500000, 'price_currency_code'...","[{'price_amount': None, 'price_currency_code':...",,http://blog.digg.com/?feed=rss2,http://blog.digg.com/,news,"[{'competitor': {'

In [8]:
# Todo el dataset
df.shape

(18801, 42)

### Filtro seleccionado: 

Filtramos las empresas que cumplen: son de videjuegos, más de 500 empleados, abiertas antes de 2012, y no en USA por problemas políticos


In [39]:
companies = collection_companies.find({"$and":[
                                {"offices": {"$exists": True}},
                                {"offices": {"$ne": None}}, 
                                {"category_code": {"$exists": True}},
                                {"category_code": {"$ne": None}}, 
                                {"founded_year": {"$exists": True}}, 
                                {"founded_year": {"$gte": 2003}},
                                {"deadpooled_year": None},
                                {"number_of_employees": {"$exists": True}},
                                {"number_of_employees": {"$gte": 10}},
                                {"total_money_raised": {"$exists": True}},
                                {"total_money_raised":{"$ne":None}},
                                {"total_money_raised": {"$not":{"$size":0}}}, 
                                {"$or": [
                                    {"total_money_raised": {"$gte": 1_000_000}},
                                    {"category_code": "design" } ,
                                    {"category_code": "web" } , 
                                    {"category_code": "software" } , 
                                    {"category_code": "games_video" } , 
                                    {"category_code": "mobile" } , 
                                    {"category_code": "enterprise" } ,   
                                    {"category_code": "analytics" } ,
                                    {'category_code': "search"},
                                    {'category_code': "network_hosting"} ,   
                                    {"category_code": "photo_video" } ,   
                                    ]} ,
                                      
                                ]
                                },
                                
                                # descartamos elementos que no interesan
                                {"_id": 0, "crunchbase_url": 0, "products": 0, 
                                 "acquisition": 0, "acquisitions": 0, "video_embeds": 0, 
                                 "screenshots": 0, "external_links": 0, "partners": 0, 
                                 "image": 0, 
                                 'deadpooled_day': 0, 'deadpooled_month': 0, 
                                 'deadpooled_url': 0, 'deadpooled_year': 0,
                                }
                                     )


In [40]:

lst = list(companies)
len(lst)

1683

In [41]:
df = pd.DataFrame(lst)
df.head(2)

,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,description,email_address,founded_day,founded_month,founded_year,funding_rounds,homepage_url,investments,ipo,milestones,name,number_of_employees,offices,overview,permalink,phone_number,providerships,relationships,tag_list,total_money_raised,twitter_username,updated_at
0,,http://blog.geni.com/index.rdf,http://blog.geni.com,web,"[{'competitor': {'name': 'Ancestry', 'permalin...",Thu May 31 19:52:34 UTC 2007,Geneology social network site,,1.0,6.0,2006,"[{'id': 6, 'round_code': 'a', 'source_url': ''...",http://www.geni.com,[],None,"[{'id': 15460, 'description': 'Announced hire ...",Geni,18,"[{'description': 'Headquarters', 'address1': '...",<p>Geni is an online community of casual and e...,geni,,[],"[{'is_past': False, 'title': 'CEO', 'person': ...","geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012
1,,http://blog.powerset.com/atom.xml,http://blog.powerset.com/,search,"[{'competitor': {'name': 'Hakia', 'permalink':...",Mon Jun 11 19:33:13 UTC 2007,,info@powerset.com,NaN,10.0,2006,"[{'id': 22, 'round_code': 'a', 'source_url': '...",http://powerset.com,[],None,[],Powerset,60,"[{'description': None, 'address1': '475 Branna...",<p>Powerset is a search engine focused on natu...,powerset,415-848-7000,"[{'title': '', 'is_past': False, 'provider': {...","[{'is_past': False, 'title': 'Founder / Produc...","searchengine, google, techcrunch40, natural-la...",$22.5M,Powerset,Sat Mar 16 06:13:05 UTC 2013


In [42]:
columns_to_count = ['competitions', 'funding_rounds', 'investments', 'offices' ,'providerships', 
                    'relationships', 'milestones']

def count_elements(x): 
    try: 
        return len(x)
    except: 
        return 0

def clean_lists(dataframe, cols): 
    for column in cols: 
        dataframe[column] = dataframe[column].apply(count_elements)
    return dataframe

df = clean_lists(df, columns_to_count)
df.head()

,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,description,email_address,founded_day,founded_month,founded_year,funding_rounds,homepage_url,investments,ipo,milestones,name,number_of_employees,offices,overview,permalink,phone_number,providerships,relationships,tag_list,total_money_raised,twitter_username,updated_at
0,,http://blog.geni.com/index.rdf,http://blog.geni.com,web,2,Thu May 31 19:52:34 UTC 2007,Geneology social network site,,1.0,6.0,2006,3,http://www.geni.com,0,None,1,Geni,18,1,<p>Geni is an online community of casual and e...,geni,,0,13,"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012
1,,http://blog.powerset.com/atom.xml,http://blog.powerset.com/,search,13,Mon Jun 11 19:33:13 UTC 2007,,info@powerset.com,NaN,10.0,2006,3,http://powerset.com,0,None,0,Powerset,60,1,<p>Powerset is a search engine focused on natu...,powerset,415-848-7000,1,20,"searchengine, google, techcrunch40, natural-la...",$22.5M,Powerset,Sat Mar 16 06:13:05 UTC 2013
2,,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,9,Thu Jun 14 03:42:20 UTC 2007,Life long learning content,jason@mahalo.com,1.0,3.0,2007,2,http://mahalo.com,0,None,0,Mahalo,40,1,<p>Mahalo is a human powered search engine fou...,mahalo,,0,13,"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013
3,,http://feeds.feedburner.com/kyte_blog,http://kyte.com/blog,games_video,10,Thu Jun 14 18:26:11 UTC 2007,Online & Mobile Video Platform,info@kyte.com,1.0,12.0,2006,4,http://www.kyte.com,0,None,0,Kyte,40,1,<p>Kyte is the online and mobile video platfor...,kyte,,0,11,"video, mobile, iphone-app, video-platform, mob...",$23.4M,kyte,Mon Oct 28 09:34:37 UTC 2013
4,,,,mobile,0,Sat Jun 16 05:53:38 UTC 2007,Voice and Mobile Search,pr@jinglenetworks.com,1.0,9.0,2005,7,http://jinglenetworks.com,0,None,2,Jingle Networks,35,1,<p>Jingle Networks is the leading provider of ...,jingle-networks,212-481-4114 x237,2,11,"directoryassistance, advertising, mobile",$88.7M,Marchex,Mon Jun 03 05:26:22 UTC 2013


In [43]:
COINS = {
        '$': 'DOL', 
        '€': 'EUR', 
        'C$': 'CAD', 
        'C': 'CAD', 
        '£': 'GBP', 
        '¥': 'JPY', 
        'kr': 'SEK'
    }
def get_coin(x, coins=COINS): 
    
    for coin, name in coins.items(): 
        if coin in x: 
            return name
    else: 
        return 'unknown'
        
df['coin'] = df.total_money_raised.apply(get_coin)
df.coin.value_counts()

DOL    1625
EUR      43
GBP      12
JPY       2
SEK       1
Name: coin, dtype: int64

In [44]:
MULTIPLES = {'B': 1e9, 'M': 1e6, 'k': 1e3}
def get_value(x, coins=COINS, multiples=MULTIPLES): 
    try: 
        # delete coin simbol
        for k in coins.keys(): 
            x = x.replace(k, "")

        # multiply if necesary. 
        for item, number in multiples.items(): 
            if item in x: 
                x = x.replace(item, "")
                return float(x) * number
        return float(x)
    except: 
        return x




get_value('$1.44M')

1440000.0

In [45]:
df['total_amount_raised'] = df.total_money_raised.apply(get_value)
df.head()

,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,description,email_address,founded_day,founded_month,founded_year,funding_rounds,homepage_url,investments,ipo,milestones,name,number_of_employees,offices,overview,permalink,phone_number,providerships,relationships,tag_list,total_money_raised,twitter_username,updated_at,coin,total_amount_raised
0,,http://blog.geni.com/index.rdf,http://blog.geni.com,web,2,Thu May 31 19:52:34 UTC 2007,Geneology social network site,,1.0,6.0,2006,3,http://www.geni.com,0,None,1,Geni,18,1,<p>Geni is an online community of casual and e...,geni,,0,13,"geni, geneology, social, family, genealogy",$16.5M,geni,Wed Oct 10 14:01:29 UTC 2012,DOL,16500000.0
1,,http://blog.powerset.com/atom.xml,http://blog.powerset.com/,search,13,Mon Jun 11 19:33:13 UTC 2007,,info@powerset.com,NaN,10.0,2006,3,http://powerset.com,0,None,0,Powerset,60,1,<p>Powerset is a search engine focused on natu...,powerset,415-848-7000,1,20,"searchengine, google, techcrunch40, natural-la...",$22.5M,Powerset,Sat Mar 16 06:13:05 UTC 2013,DOL,22500000.0
2,,http://blog.mahalo.com/feed/,http://blog.mahalo.com/,web,9,Thu Jun 14 03:42:20 UTC 2007,Life long learning content,jason@mahalo.com,1.0,3.0,2007,2,http://mahalo.com,0,None,0,Mahalo,40,1,<p>Mahalo is a human powered search engine fou...,mahalo,,0,13,"search, search-engine, human-powered-search, aaa",$21M,MahaloDotCom,Fri May 17 04:34:19 UTC 2013,DOL,21000000.0
3,,http://feeds.feedburner.com/kyte_blog,http://kyte.com/blog,games_video,10,Thu Jun 14 18:26:11 UTC 2007,Online & Mobile Video Platform,info@kyte.com,1.0,12.0,2006,4,http://www.kyte.com,0,None,0,Kyte,40,1,<p>Kyte is the online and mobile video platfor...,kyte,,0,11,"video, mobile, iphone-app, video-platform, mob...",$23.4M,kyte,Mon Oct 28 09:34:37 UTC 2013,DOL,23400000.0
4,,,,mobile,0,Sat Jun 16 05:53:38 UTC 2007,Voice and Mobile Search,pr@jinglenetworks.com,1.0,9.0,2005,7,http://jinglenetworks.com,0,None,2,Jingle Networks,35,1,<p>Jingle Networks is the leading provider of ...,jingle-networks,212-481-4114 x237,2,11,"directoryassistance, advertising, mobile",$88.7M,Marchex,Mon Jun 03 05:26:22 UTC 2013,DOL,88700000.0


In [47]:
df.dtypes

alias_list              object
blog_feed_url           object
blog_url                object
category_code           object
competitions             int64
created_at              object
description             object
email_address           object
founded_day            float64
founded_month          float64
founded_year             int64
funding_rounds           int64
homepage_url            object
investments              int64
ipo                     object
milestones               int64
name                    object
number_of_employees      int64
offices                  int64
overview                object
permalink               object
phone_number            object
providerships            int64
relationships            int64
tag_list                object
total_money_raised      object
twitter_username        object
updated_at              object
coin                    object
total_amount_raised    float64
dtype: object

In [ ]:
df.head(1)

In [63]:
from datetime import datetime

def create_date(day, month, year): 
    print(day, month, year)
    if not day: 
        day = 1
    if not month: 
        month = 1
    day, month, year = int(day), int(month), int(year)
    date_str = '{}-{}-{}'.format(day, month, year)
    return datetime.strptime(date_str, '%d-%m-%Y')
    
    
create_date(None, 12.0, 2014)
    

None 12.0 2014


datetime.datetime(2014, 12, 1, 0, 0)

In [64]:
df['founded_date'] = df.apply(lambda x: create_date(x['founded_day'], x['founded_month'], x['founded_year']), 
                             axis=1)
df.head()

1.0 6.0 2006
nan 10.0 2006
1.0 6.0 2006
nan 10.0 2006


ValueError: ('cannot convert float NaN to integer', 'occurred at index 1')